This is a crawl for a site that has "for sale by owner" listings.  Unfortunately, the volume of listings is to low to be particularly interesting for me.  I did this anyway to keep an eye on the data because a single transaction (the absolute best one) would still be worth the investment of time.

In [77]:
import requests
import BeautifulSoup as bsoup
import pandas as pd
import time
import datetime

In [72]:
def extract_listings(txt):
    plistings = []
    s = bsoup.BeautifulSoup(txt)
    listings = s.findAll('div', {'class': 'estate-info'})
    for listing in listings:
        price = listing.find('div', {'class': 'estateSummary-price mix-estateSummary_SM-price_sm'}).text
        title = listing.find('div', {'class': 'estateSummary-title mix-estateSummary_SM-title_sm'}).text
        address = listing.find('div', {'class': 'estateSummary-address'}).text
        elems = listing.findAll('div', {'class': 'estateSummary-list'})
        beds = 0
        baths = 0
        sqft = 0
        htype = ''
        lastUpdated = listing.find('em', {'class': 'highlight-text isHiddenSM'})
        if lastUpdated is None:
            lastUpdated = ''
        else:
            lastUpdated = lastUpdated.text.replace('Last updated ', '')
        for elem in elems:
            elems2 = elem.findAll('div')
            for elem2 in elems2:
                txt = elem2.text
                if txt.find('Beds') > 0:
                    beds = float(txt[0:txt.find('Beds')].strip())
                elif txt.find('Baths') > 0:
                    baths = float(txt[0:txt.find('Baths')].strip())
                elif txt.find('Sqft') > 0:
                    sqft = int(txt[0:txt.find('Sqft')].replace(',', '').strip())
                else:
                    htype = txt
        plisting = {'price': price, 'title': title, 'lastUpdated': lastUpdated, 'address': address, 'beds': beds, 'baths': baths, 'sqft': sqft, 'htype': htype}
        plistings.append(plisting)
    return plistings

In [73]:
pnum = 1
test = -1
plistings = []
while test == -1:
    r = requests.get('http://www.forsalebyowner.com/search/list/los-angeles-california/house,condo-types/' + str(pnum) + '-page/proximity,desc-sort')
    time.sleep(.5)
    test = r.text.find('Your search did not yield any results.')
    if test == -1:
        plistings.extend(extract_listings(r.text))
    pnum += 1
df = pd.DataFrame(plistings)

In [74]:
len(plistings)

23

In [75]:
df

,address,baths,beds,htype,lastUpdated,price,sqft,title
0,"7719 Minstrel Ave,\n West H...",2.00,3,Single Family,2 days ago,"$559,000",1293,Beautiful Single Story Renovation NOW READY!!!
1,"10650 Holman Ave Unit Apt 307,\n ...",2.00,2,Condo,3 weeks ago,"$739,000",1411,Reduced! Westwood Santa Monica - Cozy Top floo...
2,"4200 Don Luis Drive,\n Los ...",2.00,2,Single Family,4 months ago,"$699,000",1961,HOUSE SOLD 8/4/15 --- Baldwin Hills - 2bd / 2ba
3,"4435 Colfax Ave Unit 104,\n ...",2.50,3,Condo,2 months ago,"$699,000",1726,Studio City Lovely Mediterranean town-home
4,"12329 Huston Street,\n Vall...",3.00,3,Single Family,a week ago,"$1,475,000",2580,"VALLEY VILLAGE, 1/2 ACRE, ESTATE LIKE GROUNDS !"
5,"5117 Rubio Ave,\n Encino",3.00,4,Single Family,a day ago,"$1,999,999",4354,For Sale By Owner
6,"6835 Nestle Ave,\n Reseda",2.00,3,Single Family,4 months ago,"$475,000",1209,Beautiful Neat Pool Home on Large Lot - 3 Bed/...
7,"8830 Chimineas ave,\n North...",2.00,3,Single Family,,"$699,000",2002,complete remodeled home for sale in northridge
8,"6112 Dill Place,\n Woodland...",2.75,3,Single Family,2 months ago,"$1,260,000",2526,Key &amp; Jonsten Realty
9,"6170 Tony Ave,\n Woodland H...",2.00,4,Single Family,5 months ago,"$574,000",1404,"Woodland Hills 4br, 2ba/Pool Chater Schools"


In [76]:
df.sort('price')

,address,baths,beds,htype,lastUpdated,price,sqft,title
13,"21408 Entrada,\n Topanga",3.00,3,Single Family,,"$1,235,000",1859,A Unique Topanga Gem in PRIME Location!
8,"6112 Dill Place,\n Woodland...",2.75,3,Single Family,2 months ago,"$1,260,000",2526,Key &amp; Jonsten Realty
4,"12329 Huston Street,\n Vall...",3.00,3,Single Family,a week ago,"$1,475,000",2580,"VALLEY VILLAGE, 1/2 ACRE, ESTATE LIKE GROUNDS !"
18,"1004 South Euclid Avenue,\n ...",3.00,3,Single Family,2 months ago,"$1,895,000",2758,"Must See! NEW CONSTRUCTION, Beautiful Home in ..."
20,"2517 JACKSON AVE,\n ROSEMEAD",4.00,4,Single Family,2 days ago,"$1,930,000",2621,office room can use as 5th bedroom. fishpond
5,"5117 Rubio Ave,\n Encino",3.00,4,Single Family,a day ago,"$1,999,999",4354,For Sale By Owner
11,"704 15th Street,\n Santa Mo...",2.00,2,Single Family,a month ago,"$3,070,000",1174,704 15th St. Santa Monica 90402
19,"407 E Sunset St,\n Long Beach",2.00,2,Single Family,,"$329,000",817,Beautiful 2 bedroom and 2 bath REMODELED home ...
6,"6835 Nestle Ave,\n Reseda",2.00,3,Single Family,4 months ago,"$475,000",1209,Beautiful Neat Pool Home on Large Lot - 3 Bed/...
12,"3669 Kensley Dr,\n Inglewood",3.00,4,Single Family,,"$550,000",1669,For Sale By Owner-In Litigation for short time


In [88]:
dt = datetime.datetime.now()
dt = str(dt.year) + '-' + str(dt.month).zfill(2) + '-' + str(dt.day).zfill(2)
df.to_csv('out_' + dt + '.tsv', index=False, sep='\t')